In [2]:
url = input("ENter")

In [3]:
from newspaper import Article
import requests
from newspaper.configuration import Configuration
import json

# URL of the article
# Custom configuration to set headers
config = Configuration()
config.browser_user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
)

# Create the Article object with the custom configuration
article = Article(url, config=config)

try:
    # Fetch and parse the article
    article.download()
    article.parse()
    
    # Extract content
    article_data = {
        "title": article.title,
        "authors": article.authors,
        "publication_date": str(article.publish_date) if article.publish_date else "Unknown",
        "content": article.text,
        "url": url
    }

    # Convert to JSON
    article_json = json.dumps(article_data, indent=4)
    print(article_json)

except Exception as e:
    print(f"An error occurred: {e}")

article_dict = json.loads(article_json)

# Extract the content part
content = article_dict.get("content", "Content not found")
print(content)

{
    "title": "\"100% Tariffs From India, 50% From EU\": What White House Said On Trump's Liberation Day Plans",
    "authors": [],
    "publication_date": "Unknown",
    "content": "Amid suspense over US President Donald Trump's \"Liberation Day tariff\" plans, White House spokesperson Karoline Leavitt confirmed on Monday that there won't be any exemptions on reciprocal tariffs. Listing tariffs imposed on American products by foreign nations, Ms Leavitt said that the \"unfair trade practices\" need to stop.\n\nThis comes as President Trump prepares to announce a round of reciprocal tariffs on April 2, a day he has labeled \"Liberation Day\" of America.\n\n\"The goal of Wednesday is a country-based tariff, but certainly sectoral tariffs. The President has said he's committed to implementing them, and I would leave it to him when he makes that announcement,\" Leavitt said, adding that \"It's time for reciprocity.\"\n\nHOLY SH*T \ud83d\udea8 Karoline Leavitt brings a chart and shuts dow

In [4]:
test_text = content

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM, GRU, Bidirectional
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Initialize NLP tools
nltk.download('stopwords')
nltk.download('wordnet')

class MultiModelNewsDetector:
    def __init__(self):
        self.max_len = 200
        self.max_words = 10000
        self.tokenizer = Tokenizer(num_words=self.max_words)
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Initialize models
        self.cnn_model = self.build_cnn_model()
        self.lstm_gru_model = self.build_lstm_gru_model()
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
    
    def build_cnn_model(self):
        """1D CNN for text classification"""
        model = Sequential([
            Embedding(self.max_words, 128, input_length=self.max_len),
            Conv1D(128, 5, activation='relu'),
            GlobalMaxPooling1D(),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def build_lstm_gru_model(self):
        """Hybrid LSTM-GRU model"""
        model = Sequential([
            Embedding(self.max_words, 128, input_length=self.max_len),
            Bidirectional(LSTM(64, return_sequences=True)),
            GRU(64),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def clean_text(self, text):
        """Standard text preprocessing"""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        words = text.split()
        words = [self.lemmatizer.lemmatize(word) for word in words if word not in self.stop_words]
        return ' '.join(words)
    
    def extract_article(self, url):
        """Universal news extractor"""
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Try multiple content selectors
            selectors = [
                {'class': ['article-content', 'article-body', 'content']},
                {'itemprop': 'articleBody'},
                {'class': re.compile('(post|entry)-content')},
                {'id': re.compile('content|body|main')}
            ]
            
            article_body = None
            for selector in selectors:
                article_body = soup.find('div', selector)
                if article_body: break
            
            if not article_body:
                article_body = soup  # Fallback to whole page
            
            paragraphs = article_body.find_all('p')
            article_text = ' '.join([p.get_text().strip() for p in paragraphs if p.get_text().strip()])
            
            if len(article_text) < 100:
                return None, "Insufficient article text"
                
            return self.clean_text(article_text), None
            
        except Exception as e:
            return None, f"Extraction error: {str(e)}"
    
    def prepare_input(self, text, model_type='cnn'):
        """Prepare input for different models"""
        if model_type == 'bert':
            return self.bert_tokenizer(
                text,
                padding='max_length',
                truncation=True,
                max_length=self.max_len,
                return_tensors='tf'
            )
        else:
            sequences = self.tokenizer.texts_to_sequences([text])
            return pad_sequences(sequences, maxlen=self.max_len)
    
    def predict_with_all_models(self, url):
        """Run prediction with all three models"""
        article_text, error = self.extract_article(url)
        if error:
            return {"error": error}
        
        # For demo, we'll fit tokenizer on the fly (in production, use pre-trained)
        self.tokenizer.fit_on_texts([article_text])
        
        # CNN Prediction
        cnn_input = self.prepare_input(article_text, 'cnn')
        cnn_pred = self.cnn_model.predict(cnn_input)[0][0]
        
        # LSTM-GRU Prediction
        lstm_gru_input = self.prepare_input(article_text, 'lstm_gru')
        lstm_gru_pred = self.lstm_gru_model.predict(lstm_gru_input)[0][0]
        
        # BERT Prediction
        bert_input = self.prepare_input(article_text, 'bert')
        bert_output = self.bert_model(bert_input)
        bert_pred = tf.sigmoid(bert_output.logits).numpy()[0][0]
        
        def format_pred(pred):
            label = 'Real' if pred > 0.5 else 'Fake'
            confidence = pred if pred > 0.5 else 1 - pred
            return {'label': label, 'confidence': float(confidence * 100)}
        
        return {
            'url': url,
            'models': {
                '1D_CNN': format_pred(cnn_pred),
                'LSTM_GRU': format_pred(lstm_gru_pred),
                'BERT': format_pred(bert_pred)
            },
            'ensemble_prediction': self.ensemble_prediction([cnn_pred, lstm_gru_pred, bert_pred])
        }
    
    def ensemble_prediction(self, predictions):
        """Combine predictions from all models"""
        avg_pred = np.mean(predictions)
        label = 'Real' if avg_pred > 0.5 else 'Fake'
        confidence = avg_pred if avg_pred > 0.5 else 1 - avg_pred
        return {'label': label, 'confidence': float(confidence * 100)}

# Example Usage
if __name__ == "__main__":
    detector = MultiModelNewsDetector()
    
    # Example news URL (replace with actual URL)
    test_url = url

    print("\nMulti-Model News Authenticity Analysis")
    print("===================================")
    print(f"Analyzing: {test_url}\n")
    
    result = detector.predict_with_all_models(test_url)
    
    if 'error' in result:
        print(f"Error: {result['error']}")
    else:
        print("Individual Model Results:")
        for model_name, pred in result['models'].items():
            print(f"{model_name:>8}: {pred['label']} ({pred['confidence']:.1f}%)")
        
        print(f"\nEnsemble Prediction: {result['ensemble_prediction']['label']} "
              f"({result['ensemble_prediction']['confidence']:.1f}% confidence)")
        

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\liyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Multi-Model News Authenticity Analysis
Analyzing: https://www.ndtv.com/world-news/100-tariffs-from-india-50-from-eu-what-white-house-said-on-donald-trumps-liberation-day-plans-8057806

1/1 [==============================] - 1s 968ms/step
Individual Model Results:
  1D_CNN: Fake (50.2%)
LSTM_GRU: Real (50.1%)
    BERT: Real (60.0%)

Ensemble Prediction: Real (53.3% confidence)

Note: This demo uses untrained models. For production use:
- Train each model on labeled fake/real news data
- Save model weights and tokenizers
- Consider using pre-trained embeddings for CNN/LSTM-GRU


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM, GRU, Bidirectional
from tensorflow.keras.models import Sequential, save_model
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize NLP tools
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

class FakeNewsDetectorSystem:
    def __init__(self, max_len=200, max_words=10000):
        self.max_len = max_len
        self.max_words = max_words
        self.tokenizer = Tokenizer(num_words=self.max_words)
        self.models = {
            'cnn': None,
            'lstm_gru': None,
            'bert': None
        }
    
    def clean_text(self, text):
        """Preprocess text for all models"""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        words = text.split()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
        return ' '.join(words)
    
    def load_and_prepare_data(self, true_path, fake_path, test_size=0.2):
        """Load and preprocess dataset"""
        # Load data
        true_df = pd.read_csv(true_path)
        fake_df = pd.read_csv(fake_path)
        
        # Label and combine
        true_df['label'] = 1  # Real news
        fake_df['label'] = 0  # Fake news
        df = pd.concat([true_df, fake_df]).sample(frac=1).reset_index(drop=True)
        
        # Clean text
        df['clean_text'] = df['text'].apply(self.clean_text)
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            df['clean_text'], df['label'], test_size=test_size, random_state=42
        )
        
        # Tokenize for CNN/LSTM-GRU
        self.tokenizer.fit_on_texts(X_train)
        X_train_seq = pad_sequences(self.tokenizer.texts_to_sequences(X_train), maxlen=self.max_len)
        X_test_seq = pad_sequences(self.tokenizer.texts_to_sequences(X_test), maxlen=self.max_len)
        
        return X_train_seq, X_test_seq, y_train, y_test, X_train, X_test
    
    def build_cnn_model(self):
        """1D CNN architecture"""
        model = Sequential([
            Embedding(self.max_words, 128, input_length=self.max_len),
            Conv1D(128, 5, activation='relu'),
            GlobalMaxPooling1D(),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def build_lstm_gru_model(self):
        """Hybrid LSTM-GRU architecture"""
        model = Sequential([
            Embedding(self.max_words, 128, input_length=self.max_len),
            Bidirectional(LSTM(64, return_sequences=True)),
            GRU(64),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def train_models(self, X_train, y_train, X_test, y_test, epochs=5, batch_size=64):
        """Train all three model architectures"""
        # Train CNN
        print("Training CNN model...")
        self.models['cnn'] = self.build_cnn_model()
        self.models['cnn'].fit(X_train, y_train, 
                              validation_data=(X_test, y_test),
                              epochs=epochs, batch_size=batch_size)
        
        # Train LSTM-GRU
        print("\nTraining LSTM-GRU model...")
        self.models['lstm_gru'] = self.build_lstm_gru_model()
        self.models['lstm_gru'].fit(X_train, y_train,
                                   validation_data=(X_test, y_test),
                                   epochs=epochs, batch_size=batch_size)
        
        # Initialize BERT components (fine-tuning requires separate setup)
        print("\nInitializing BERT model (requires separate fine-tuning script)")
        self.models['bert'] = {
            'tokenizer': BertTokenizer.from_pretrained('bert-base-uncased'),
            'model': TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
        }
    
    def save_models(self, save_dir='saved_models'):
        """Save all model components for production"""
        import os
        os.makedirs(save_dir, exist_ok=True)
        
        # Save CNN model
        save_model(self.models['cnn'], f'{save_dir}/cnn_model.h5')
        
        # Save LSTM-GRU model
        save_model(self.models['lstm_gru'], f'{save_dir}/lstm_gru_model.h5')
        
        # Save BERT components
        self.models['bert']['model'].save_pretrained(f'{save_dir}/bert_model')
        self.models['bert']['tokenizer'].save_pretrained(f'{save_dir}/bert_tokenizer')
        
        # Save tokenizer
        with open(f'{save_dir}/tokenizer.pkl', 'wb') as f:
            pickle.dump(self.tokenizer, f)
        
        print(f"All models saved to {save_dir} directory")
    
    def load_models(self, save_dir='saved_models'):
        """Load pre-trained models"""
        # Load CNN
        self.models['cnn'] = tf.keras.models.load_model(f'{save_dir}/cnn_model.h5')
        
        # Load LSTM-GRU
        self.models['lstm_gru'] = tf.keras.models.load_model(f'{save_dir}/lstm_gru_model.h5')
        
        # Load BERT
        self.models['bert'] = {
            'tokenizer': BertTokenizer.from_pretrained(f'{save_dir}/bert_tokenizer'),
            'model': TFBertForSequenceClassification.from_pretrained(f'{save_dir}/bert_model')
        }
        
        # Load tokenizer
        with open(f'{save_dir}/tokenizer.pkl', 'rb') as f:
            self.tokenizer = pickle.load(f)
        
        print("All models loaded successfully")

# Example Usage for Training
if __name__ == "__main__":
    # Initialize system
    detector = FakeNewsDetectorSystem()
    
    # Paths to your dataset (example using ISOT dataset)
    TRUE_DATA_PATH = "True.csv"
    FAKE_DATA_PATH = "Fake.csv"
    
    # Load and prepare data
    print("Loading and preprocessing data...")
    X_train_seq, X_test_seq, y_train, y_test, X_train_raw, X_test_raw = detector.load_and_prepare_data(
        TRUE_DATA_PATH, FAKE_DATA_PATH
    )
    
    # Train models
    print("\nTraining models...")
    detector.train_models(X_train_seq, y_train, X_test_seq, y_test, epochs=3)
    
    # Save models for production
    print("\nSaving trained models...")
    detector.save_models()
    
    # To load models later:
    # new_detector = FakeNewsDetectorSystem()
    # new_detector.load_models()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\liyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading and preprocessing data...


In [8]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

def bert_fine_tuning(train_texts, train_labels, val_texts, val_labels):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Convert data to BERT format
    train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=200)
    val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=200)
    
    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(train_encodings),
        train_labels
    ))
    val_dataset = tf.data.Dataset.from_tensor_slices((
        dict(val_encodings),
        val_labels
    ))
    
    # Load and fine-tune BERT
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    
    model.fit(train_dataset.shuffle(1000).batch(16),
              epochs=2,
              batch_size=16,
              validation_data=val_dataset.batch(16))
    
    return model, tokenizer

In [10]:
class FakeNewsDetectorSystem:
    # ... (keep all existing methods) ...
    
    def save_models(self, save_dir='saved_models'):
        """Save all model components for production"""
        import os
        os.makedirs(save_dir, exist_ok=True)
        
        # Save CNN model and weights
        save_model(self.models['cnn'], f'{save_dir}/cnn_model.h5')
        self.models['cnn'].save_weights(f'{save_dir}/cnn_weights.h5')
        
        # Save LSTM-GRU model and weights
        save_model(self.models['lstm_gru'], f'{save_dir}/lstm_gru_model.h5')
        self.models['lstm_gru'].save_weights(f'{save_dir}/lstm_gru_weights.h5')
        
        # Save BERT components
        bert_dir = f'{save_dir}/bert_model'
        os.makedirs(bert_dir, exist_ok=True)
        self.models['bert']['model'].save_pretrained(bert_dir)
        self.models['bert']['tokenizer'].save_pretrained(f'{save_dir}/bert_tokenizer')
        
        # Save tokenizer
        with open(f'{save_dir}/tokenizer.pkl', 'wb') as f:
            pickle.dump(self.tokenizer, f)
        
        print(f"All models and weights saved to {save_dir} directory")
    
    def save_weights_only(self, save_dir='saved_weights'):
        """Save only the weights of the models (not full architecture)"""
        import os
        os.makedirs(save_dir, exist_ok=True)
        
        # Save CNN weights
        self.models['cnn'].save_weights(f'{save_dir}/cnn_weights.h5')
        
        # Save LSTM-GRU weights
        self.models['lstm_gru'].save_weights(f'{save_dir}/lstm_gru_weights.h5')
        
        # Save BERT weights (special handling for transformers)
        bert_dir = f'{save_dir}/bert_weights'
        os.makedirs(bert_dir, exist_ok=True)
        self.models['bert']['model'].save_weights(f'{bert_dir}/bert_weights.h5')
        
        print(f"Model weights saved to {save_dir} directory")
    
    def load_weights_only(self, save_dir='saved_weights'):
        """Load only the weights into existing model architectures"""
        # Load CNN weights
        self.models['cnn'].load_weights(f'{save_dir}/cnn_weights.h5')
        
        # Load LSTM-GRU weights
        self.models['lstm_gru'].load_weights(f'{save_dir}/lstm_gru_weights.h5')
        
        # Load BERT weights
        self.models['bert']['model'].load_weights(f'{save_dir}/bert_weights/bert_weights.h5')
        
        print("All model weights loaded successfully")

In [2]:
# 1. Keep your original FakeNewsDetectorSystem for training/saving
# 2. Add this as a new class at the end (modified slightly):

class FakeNewsPredictor:
    def __init__(self, model_dir='saved_models'):
        """Initialize with pre-trained models"""
        # Load your saved models here
        self.detector = FakeNewsDetectorSystem()
        self.detector.load_models(model_dir)
        
        # Keep the web scraping and prediction methods from the new code
        self.max_len = 200
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
    
    # Add all the web scraping and prediction methods from the new code
    def extract_article(self, url): ...
    def predict_with_all_models(self, url): ...
    # etc.

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM, GRU, Bidirectional
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

# Initialize NLP tools
nltk.download('stopwords')
nltk.download('wordnet')

class FakeNewsDetector:
    def __init__(self, max_len=200, max_words=10000):
        self.max_len = max_len
        self.max_words = max_words
        self.tokenizer = Tokenizer(num_words=self.max_words)
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.models = {
            'cnn': None,
            'lstm_gru': None,
            'bert': None
        }
    
    def clean_text(self, text):
        """Preprocess text by removing special chars, numbers, and lemmatizing"""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        words = text.split()
        words = [self.lemmatizer.lemmatize(word) for word in words if word not in self.stop_words]
        return ' '.join(words)
    
    def load_data(self, true_path, fake_path):
        """Load and combine true and fake news datasets"""
        true_df = pd.read_csv(true_path)
        fake_df = pd.read_csv(fake_path)
        
        true_df['label'] = 1  # 1 for real news
        fake_df['label'] = 0  # 0 for fake news
        
        df = pd.concat([true_df, fake_df]).sample(frac=1).reset_index(drop=True)
        df['clean_text'] = df['text'].apply(self.clean_text)
        
        return df
    
    def prepare_datasets(self, df, test_size=0.2):
        """Split data into train/test sets and tokenize"""
        X_train, X_test, y_train, y_test = train_test_split(
            df['clean_text'], df['label'], test_size=test_size, random_state=42
        )
        
        # Tokenize text
        self.tokenizer.fit_on_texts(X_train)
        X_train_seq = pad_sequences(self.tokenizer.texts_to_sequences(X_train), maxlen=self.max_len)
        X_test_seq = pad_sequences(self.tokenizer.texts_to_sequences(X_test), maxlen=self.max_len)
        
        return X_train_seq, X_test_seq, y_train, y_test
    
    def build_cnn_model(self):
        """Create 1D CNN model architecture"""
        model = Sequential([
            Embedding(self.max_words, 128, input_length=self.max_len),
            Conv1D(128, 3, activation='relu'),
            GlobalMaxPooling1D(),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def build_lstm_gru_model(self):
        """Create hybrid LSTM-GRU model architecture"""
        model = Sequential([
            Embedding(self.max_words, 128, input_length=self.max_len),
            Bidirectional(LSTM(64, return_sequences=True)),
            GRU(64),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def initialize_bert(self):
        """Initialize BERT model components"""
        self.models['bert'] = {
            'tokenizer': BertTokenizer.from_pretrained('bert-base-uncased'),
            'model': TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
        }
    
    def train_models(self, X_train, y_train, X_test, y_test, epochs=5, batch_size=64):
        """Train all three model architectures"""
        early_stopping = EarlyStopping(patience=2, restore_best_weights=True)
        
        # Train CNN
        print("Training CNN model...")
        self.models['cnn'] = self.build_cnn_model()
        self.models['cnn'].fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping]
        )
        
        # Train LSTM-GRU
        print("\nTraining LSTM-GRU model...")
        self.models['lstm_gru'] = self.build_lstm_gru_model()
        self.models['lstm_gru'].fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping]
        )
        
        # Initialize BERT (requires separate fine-tuning)
        print("\nInitializing BERT model...")
        self.initialize_bert()
    
    def save_models(self, save_dir='saved_models'):
        """Save all model components to disk"""
        os.makedirs(save_dir, exist_ok=True)
        
        # Save CNN model
        save_model(self.models['cnn'], f'{save_dir}/cnn_model.h5')
        
        # Save LSTM-GRU model
        save_model(self.models['lstm_gru'], f'{save_dir}/lstm_gru_model.h5')
        
        # Save BERT components
        self.models['bert']['model'].save_pretrained(f'{save_dir}/bert_model')
        self.models['bert']['tokenizer'].save_pretrained(f'{save_dir}/bert_tokenizer')
        
        # Save tokenizer
        with open(f'{save_dir}/tokenizer.pkl', 'wb') as f:
            pickle.dump(self.tokenizer, f)
        
        print(f"All models saved to {save_dir} directory")
    
    def load_models(self, save_dir='saved_models'):
        """Load pre-trained models from disk"""
        # Load CNN
        self.models['cnn'] = tf.keras.models.load_model(f'{save_dir}/cnn_model.h5')
        
        # Load LSTM-GRU
        self.models['lstm_gru'] = tf.keras.models.load_model(f'{save_dir}/lstm_gru_model.h5')
        
        # Load BERT
        self.models['bert'] = {
            'tokenizer': BertTokenizer.from_pretrained(f'{save_dir}/bert_tokenizer'),
            'model': TFBertForSequenceClassification.from_pretrained(f'{save_dir}/bert_model')
        }
        
        # Load tokenizer
        with open(f'{save_dir}/tokenizer.pkl', 'rb') as f:
            self.tokenizer = pickle.load(f)
        
        print("All models loaded successfully")
    
    def predict(self, text, model_type='cnn'):
        """Make prediction on new text"""
        cleaned_text = self.clean_text(text)
        
        if model_type == 'bert':
            inputs = self.models['bert']['tokenizer'](
                cleaned_text,
                return_tensors='tf',
                padding='max_length',
                truncation=True,
                max_length=self.max_len
            )
            outputs = self.models['bert']['model'](inputs)
            pred = tf.sigmoid(outputs.logits).numpy()[0][0]
        else:
            sequence = self.tokenizer.texts_to_sequences([cleaned_text])
            padded_seq = pad_sequences(sequence, maxlen=self.max_len)
            pred = self.models[model_type].predict(padded_seq)[0][0]
        
        return {
            'prediction': 'Real' if pred > 0.5 else 'Fake',
            'confidence': float(pred if pred > 0.5 else 1 - pred),
            'raw_score': float(pred)
        }

# Main execution
if __name__ == "__main__":
    # Initialize detector
    detector = FakeNewsDetector()
    
    # Paths to your dataset files
    TRUE_DATA_PATH = "True.csv"
    FAKE_DATA_PATH = "Fake.csv"
    
    # Load and prepare data
    print("Loading and preprocessing data...")
    df = detector.load_data(TRUE_DATA_PATH, FAKE_DATA_PATH)
    X_train, X_test, y_train, y_test = detector.prepare_datasets(df)
    
    # Train models
    print("\nTraining models...")
    detector.train_models(X_train, y_train, X_test, y_test, epochs=5)
    
    # Save models
    print("\nSaving models...")
    detector.save_models()
    
    # Example prediction

    print(f"\nTesting prediction now....\n")
    
    # Load models (simulating a fresh start)
    new_detector = FakeNewsDetector()
    new_detector.load_models()
    
    # Make predictions with different models
    print("\nModel Predictions:")
    for model_name in ['cnn', 'lstm_gru', 'bert']:
        result = new_detector.predict(test_text, model_type=model_name)
        print(f"{model_name.upper():<8}: {result['prediction']} (confidence: {result['confidence']*100:.1f}%)")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\liyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading and preprocessing data...

Training models...
Training CNN model...
Epoch 1/5
562/562 [==============================] - 69s 122ms/step - loss: 0.0892 - accuracy: 0.9635 - val_loss: 0.0174 - val_accuracy: 0.9945
Epoch 2/5
562/562 [==============================] - 86s 153ms/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 0.0172 - val_accuracy: 0.9937
Epoch 3/5
562/562 [==============================] - 87s 155ms/step - loss: 7.3922e-04 - accuracy: 0.9999 - val_loss: 0.0169 - val_accuracy: 0.9941
Epoch 4/5
562/562 [==============================] - 84s 150ms/step - loss: 3.8837e-04 - accuracy: 1.0000 - val_loss: 0.0170 - val_accuracy: 0.9938
Epoch 5/5
562/562 [==============================] - 77s 138ms/step - loss: 3.1641e-04 - accuracy: 1.0000 - val_loss: 0.0174 - val_accuracy: 0.9941

Training LSTM-GRU model...
Epoch 1/5
562/562 [==============================] - 321s 558ms/step - loss: 0.1004 - accuracy: 0.9634 - val_loss: 0.0569 - val_accuracy: 0.9849
Epoch 2/5
562/562 [

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Saving models...
All models saved to saved_models directory

Testing prediction now....



Some layers from the model checkpoint at saved_models/bert_model were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at saved_models/bert_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


All models loaded successfully

Model Predictions:
1/1 [==============================] - 0s 214ms/step
CNN     : Real (confidence: 100.0%)
1/1 [==============================] - 1s 1s/step
LSTM_GRU: Real (confidence: 100.0%)
BERT    : Real (confidence: 52.5%)
